In [24]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [25]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [26]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [27]:
 import numpy as np

In [28]:
import matplotlib.pyplot as plt

In [29]:
image_data = np.load('X_train.npy')

In [30]:
image_data_normalized = image_data 


In [63]:
marker = np.load('Y_train.npy')
marker.dtype
marker.shape
num_ones = np.count_nonzero(marker == 1)
print(num_ones)

dtype('int64')

(4160,)

2087


In [35]:
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(image_data_normalized, marker, test_size=0.3, random_state=6)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=6)


In [36]:
X_train.shape
X_val.shape
X_test.shape

(2912, 50, 50, 3)

(624, 50, 50, 3)

(624, 50, 50, 3)

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
 

In [50]:
model = Sequential()

In [51]:
model.add(Conv2D(32,(3,3), 1, activation='relu',input_shape = (50,50,3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64,(3,3), 1, activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128,(3,3), 1, activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(layers.Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
# model.add(Dense(1,activation='sigmoid', bias_initializer=tf.keras.initializers.Constant(value=-12.00)))

In [52]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)


In [53]:
from keras import backend as K

def weighted_binary_crossentropy(y_true, y_pred):
    class_weights = tf.constant([1., 2.])
    bce = K.binary_crossentropy(y_true, y_pred)
    weighted_bce = K.mean(bce * class_weights)
    return weighted_bce

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 48, 48, 32)        896       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 24, 24, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 22, 22, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 11, 11, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 9, 9, 128)         73856     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 4, 4, 128)        

In [54]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Train on 2912 samples, validate on 624 samples
Epoch 1/20
2912/2912 [==============================] - ETA: 0s - loss: 2.3531 - acc: 0.5464

C:\Users\divya\mlbasics\sample_project_1\env\Lib\site-packages\keras\src\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


2912/2912 [==============================] - 9s 3ms/sample - loss: 2.3531 - acc: 0.5464 - val_loss: 0.6578 - val_acc: 0.6250
Epoch 2/20
2912/2912 [==============================] - 9s 3ms/sample - loss: 0.6408 - acc: 0.6617 - val_loss: 0.6363 - val_acc: 0.6330
Epoch 3/20
2912/2912 [==============================] - 9s 3ms/sample - loss: 0.6249 - acc: 0.6727 - val_loss: 0.6605 - val_acc: 0.6170
Epoch 4/20
2912/2912 [==============================] - 9s 3ms/sample - loss: 0.6192 - acc: 0.6741 - val_loss: 0.6185 - val_acc: 0.6490
Epoch 5/20
2912/2912 [==============================] - 9s 3ms/sample - loss: 0.5892 - acc: 0.7012 - val_loss: 0.6014 - val_acc: 0.6875
Epoch 6/20
2912/2912 [==============================] - 9s 3ms/sample - loss: 0.6292 - acc: 0.6611 - val_loss: 0.6391 - val_acc: 0.6538
Epoch 7/20
2912/2912 [==============================] - 9s 3ms/sample - loss: 0.6012 - acc: 0.6988 - val_loss: 0.5957 - val_acc: 0.6939
Epoch 8/20
2912/2912 [==============================] - 9s 

In [55]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test accuracy:', test_acc)

Test accuracy: 0.7307692


In [56]:
x_test = np.load('X_test.npy')
yp = model.predict(X_test)
threshold = 0.1

mn = 1000
ans = 0.1

for k in range(90):
    a = 0
    b = 0
    rp = (yp > threshold).astype(int)
    
    for i in range(len(y_test)):
        if y_test[i] != rp[i]:
            if y_test[i] == 1:
                a+=1
            else:
                b+=1

    if(abs(a-b) < mn):
        ans = threshold
        mn = abs(a-b)

    threshold+=0.01

print(mn)
print(ans)

C:\Users\divya\mlbasics\sample_project_1\env\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


0
0.4700000000000003


In [57]:
predictions = model.predict(x_test)
x_test.shape
predictions.shape

(1387, 50, 50, 3)

(1387, 1)

In [58]:
predictions

array([[0.10787905],
       [0.8407686 ],
       [0.1733684 ],
       ...,
       [0.10448074],
       [0.45536417],
       [0.7984514 ]], dtype=float32)

In [60]:
import pandas as pd
threshold = 0.4700000  

rounded_predictions = (predictions > threshold).astype(int)

output_df = pd.DataFrame(rounded_predictions)

output_df.to_csv('divyansh_sinha.csv', index=False)